 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Prepare-the-data" data-toc-modified-id="Prepare-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prepare the data</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Theano-Implementation" data-toc-modified-id="Theano-Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Theano Implementation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-for-a-single-day" data-toc-modified-id="Train-for-a-single-day-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Train for a single day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-all-of-2016-with-a-rolling-window" data-toc-modified-id="Post-processing-for-all-of-2016-with-a-rolling-window-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Post processing for all of 2016 with a rolling window</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Keras-implementation" data-toc-modified-id="Keras-implementation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Keras implementation</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Predict-for-one-day" data-toc-modified-id="Predict-for-one-day-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Predict for one day</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Post-processing-for-2016" data-toc-modified-id="Post-processing-for-2016-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Post-processing for 2016</a></span></li><li><span><a href="http://localhost:8888/notebooks/EMOS_network.ipynb#Train-2015,-predict-2016" data-toc-modified-id="Train-2015,-predict-2016-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Train 2015, predict 2016</a></span></li></ul></li></ul></div>

# EMOS Network

The goal of this notebook is to build and test a network implementation of EMOS, once in pure theano and then in keras. First we will try to replicate the results of the standard global EMOS. 

The reference period is always the mean CRPS for all dates and stations in 2016.

In [119]:
# Imports
# Note that the cost function only works with the theano backend for keras
from importlib import reload
import emos_network_theano; reload(emos_network_theano)
from  emos_network_theano import EMOS_Network
from crps_loss import crps_cost_function
import utils; reload(utils)
from utils import *
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt

In [97]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'   # Where to save post-processed predictions
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Prepare the data

The interpolated raw data set contains the observations and forecasts from the 50 members for each station and each day. Using the `get_train_test_sets` function we will convert this raw dataset to a format suitable for the networks.

Here we pick a forecast date and return as the training data all previous days within the window size previous to the start of the forecast. The test data is simply the data for the chosen forecast date. The 50 member ensemble is summarized by the mean and the standard deviation. Additionally, we remove all data where the observations are missing. Finally, the inputs are scaled. For this we simply divide each feature by its maximum in the training set.

In [80]:
# Chose a random date to illustrate the algorithm
date_str = '2011-02-14'   # This is our standard date format

In [81]:
# Load training and test set
train_set, test_set = get_train_test_sets(DATA_DIR, predict_date=date_str,
                                          fclt=fclt, window_size=window_size)

train set contains 25 days
test set contains 1 days


These two sets are objects which contain all the data and some meta information.

In [85]:
train_set.feature_names

['t2m_fc_mean', 't2m_fc_std']

In [86]:
train_set.features.shape, train_set.targets.shape, train_set.date_strs.shape

((12619, 2), (12619,), (12619,))

In [87]:
test_set.features.shape, test_set.targets.shape, test_set.date_strs.shape

((503, 2), (503,), (503,))

## Theano Implementation

To start with we build the model in pure theano. The model is defined in a separate script `EMOS_network_theano.py`. The network uses a custom CRPS loss function which is defined in `crps_loss.py`.

The EMOS_Network class is build to work in a similar way to keras models. For the fitting we are using gradient descent. Since we are using the entire dataset for each update, it is not stochastic. An early stopping algorithm is built into the fitting function. It stops training if the average training CRPS of the last 5 steps is decreasing by less than a parameter delta. 

### Train for a single day

To illustrate how the model work we will use the data for our example day above and fit the model.

In [88]:
# Define some model parameters
lr = np.asarray(0.1, dtype='float32')   # The learning rate
early_stopping_delta = 1e-4   # How much the CRPS must improve before stopping
steps_max = 1000   # How many steps to fit at max

In [89]:
# Set up the theano model
model_theano = EMOS_Network()

In [90]:
# Split the features into means and standard deviation
train_set.feature_names

['t2m_fc_mean', 't2m_fc_std']

In [91]:
train_mean = train_set.features[:, 0]
train_std = train_set.features[:, 1]
test_mean = test_set.features[:, 0]
test_std = test_set.features[:, 1]

In [92]:
# Train the model for some steps
model_theano.fit(train_mean, train_std, train_set.targets, steps_max, 
                 (test_mean, test_std, test_set.targets), lr=lr, 
                 early_stopping_delta=early_stopping_delta)
# Output is the training CRPS and the test CRPS

(array(1.1374654313399257), array(0.778464882363388))

### Post processing for all of 2016 with a rolling window

To compare the network model with the standard EMOS we will run it from 1 January 2016 to 31 December 2016. When looping over the days we are not resetting the model weights for each day. This drastically reduces the training time with identical results.

In [93]:
# Get start and stop indices
date_str_start = '2016-01-01'
date_str_stop = '2017-01-01'

In [94]:
model_theano = EMOS_Network()

In [95]:
# This function loops over the days.
train_crps_list, valid_crps_list, results_df = loop_over_days(
    DATA_DIR,
    model_theano,
    date_str_start, date_str_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    verbose=0,
    model_type='EMOS_Network_theano')

100%|██████████| 366/366 [05:45<00:00,  1.14it/s]


Note that before restructuring the data preparation function, this was significantly slower. It might have to do with the creation of the meta data arrays.

In [98]:
# Let's see what the mean prediction CRPS is
np.mean(valid_crps_list)

1.0201273233365458

In [99]:
# Save the results
results_df.to_csv(results_dir + 'emos_network_rolling_window.csv')

This file is then read by the evaluation script.

## Keras implementation

Now let's build the same model in keras. This will provide a good starting point to expand the model later on.

In [100]:
# import the keras modules
# Note that the cost function only works with the theano backend
import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [101]:
# Let's build the model with Keras' functional API
# This is quite a bit easier and shorter than in theano 
def build_EMOS_Network_keras():
    mean_in = Input(shape=(1,))
    std_in = Input(shape=(1,))
    mean_out = Dense(1, activation='linear')(mean_in)
    std_out = Dense(1, activation='linear')(std_in)
    x = keras.layers.concatenate([mean_out, std_out], axis=1)
    return Model(inputs=[mean_in, std_in], outputs=x)

In [102]:
model_keras = build_EMOS_Network_keras()

In [103]:
model_keras.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
input_8 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
dense_7 (Dense)                  (None, 1)             2           input_7[0][0]                    
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 1)             2           input_8[0][0]                    
___________________________________________________________________________________________

In [104]:
# Compile the model with SGD optimizer and our custom loss function
opt = SGD(lr=0.1)  
model_keras.compile(optimizer=opt, loss=crps_cost_function, 
                    metrics=[crps_cost_function])

### Predict for one day

In keras the early stopping algorithm works slightly differently. It stops training once the training loss hasn't decreased by an amount delta in a certain number of steps (patience).

In [105]:
# This way we have the gradient descent on the whole training set just as in theano
batch_size = train_mean.shape[0]   

In [106]:
model_keras.fit([train_mean, train_std], train_set.targets, epochs=steps_max, 
                batch_size=batch_size,
                validation_data=[[test_mean, test_std], test_set.targets], 
                verbose=0,
                callbacks=[EarlyStopping(monitor='loss', 
                                         min_delta=early_stopping_delta,
                                         patience=2)])

In [107]:
# Get train and test CRPS
(model_keras.evaluate([train_mean, train_std], train_set.targets, batch_size, verbose=0), 
 model_keras.evaluate([test_mean, test_std], test_set.targets, batch_size, verbose=0))

([1.1320928811822395, 1.1320928811822393],
 [0.77120698033628332, 0.77120698033628332])

We get very similar results to the theano implementation.

### Post-processing for 2016

Same as above with the theano model.

In [108]:
model_keras = build_EMOS_Network_keras()
opt = SGD(lr=0.1)  
model_keras.compile(optimizer=opt, loss=crps_cost_function, 
                    metrics=[crps_cost_function])

In [121]:
date_str_start = '2016-01-01'
date_str_stop = '2017-01-01'
# This function loops over the days.
train_crps_list, valid_crps_list, results_df = loop_over_days(
    DATA_DIR,
    model_keras,
    date_str_start, date_str_stop, 
    window_size=window_size,
    fclt=fclt,     
    epochs_max=steps_max, 
    early_stopping_delta=early_stopping_delta, 
    lr=lr,
    verbose=0,
    model_type='EMOS_Network_keras')

100%|██████████| 366/366 [08:14<00:00,  1.09s/it]


The keras implementation is slower than the pure theano version. This could be due to the overhead of calling model.fit many many times. 

In [122]:
np.mean(train_crps_list), np.mean(valid_crps_list)

(0.99132742167771914, 1.0064540886440319)

The results are very similar to the theano implementation.

### Train 2015, predict 2016

Finally, we will train one single model on all of the 2015 data, and then post-process all of 2016 with this one model. 

In [123]:
train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']

In [124]:
# Load data sets
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates)

train set contains 365 days
test set contains 366 days


In [125]:
model_keras = build_EMOS_Network_keras()
opt = Adam(lr=0.1)  # Adam is a better SGD in a nutshell
model_keras.compile(optimizer=opt, loss=crps_cost_function)

In [126]:
# Split dataset
train_mean = train_set.features[:, 0]
train_std = train_set.features[:, 1]
test_mean = test_set.features[:, 0]
test_std = test_set.features[:, 1]

In [127]:
model_keras.fit([train_mean, train_std], train_set.targets, epochs=10, 
                batch_size=1024, 
                validation_data=[[test_mean, test_std], test_set.targets])

Train on 180849 samples, validate on 182218 samples
Epoch 1/10
180849/180849 [==============================] - 0s - loss: 3.0256 - val_loss: 1.9569
Epoch 2/10
180849/180849 [==============================] - 0s - loss: 1.3893 - val_loss: 1.0758
Epoch 3/10
180849/180849 [==============================] - 0s - loss: 1.0758 - val_loss: 1.0133
Epoch 4/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0125
Epoch 5/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0122
Epoch 6/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0112
Epoch 7/10
180849/180849 [==============================] - 0s - loss: 1.0692 - val_loss: 1.0116
Epoch 8/10
180849/180849 [==============================] - 0s - loss: 1.0693 - val_loss: 1.0121
Epoch 9/10
180849/180849 [==============================] - 0s - loss: 1.0694 - val_loss: 1.0136
Epoch 10/10
180849/180849 [==============================] - 0s - loss: 1.0

So we get a very similar CRPS compared to the 25 day rolling window. This suggests that the seasonality is not that important.

In [128]:
# Get predictions
preds = model_keras.predict([test_mean, test_std])

In [129]:
# Save predictions
results_df = create_results_df(test_set.date_strs, test_set.station_ids,
                               preds[:, 0], preds[:, 1])

In [130]:
results_df.head()

,date,mean,station_id,std
0,2016-01-01,4.507663,44.0,1.652217
1,2016-01-01,1.691526,71.0,2.386979
2,2016-01-01,0.661479,73.0,1.772558
3,2016-01-01,4.462898,78.0,1.700408
4,2016-01-01,2.021837,91.0,2.544789


In [131]:
results_df.to_csv(results_dir + 'emos_network_train_2015_pred_2016.csv')